In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def datagenerate():
    x1=np.random.uniform(0, 1)
    x2=np.random.uniform(0,1)
    b=[-np.inf,-1,-0.1,0.25,1]
    e=np.random.normal(0,0.125)
    c=[]
    for i in range(5):
        if 10*((x1-0.5)*(x2-0.5)) + e >b[i]:
            c.append(i+1)
    return [x1,x2,max(c)]
        

In [3]:
data=[]
#for i in range(100):#generating 100 sequence of instance rank pairs each of length 7000
for i in range(1):
    data.append([])
    for j in range(7000):
        data[i].append(datagenerate())

In [4]:
def widrowhoff(train,y,t,c):
    w=np.array([0 for i in range(len(np.transpose(train)))])
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,t+1):
        x=np.array(train[i-1])
        y_bart=np.dot(w,x)
        y_bar=[]
        for t in range(1,i+1):
            x=np.array(train[t-1])
            y_bart=np.dot(w,x)
            y_bar.append(y_bart)            
        l=sum([np.absolute(y[j-1]-y_bar[j-1]) for j in range(1,i+1)])/i
        print(f"loss in round {i} is ",l)
        loss.append(l)
        w=np.subtract(w,c*(np.dot(w,x)-y[i-1])*x)
    return loss,w

In [5]:
def pranking(train,y,t):
    n_feature=train.shape[1]
    n_label=len(np.unique(y))
    w=np.zeros(n_feature)
    b = np.zeros(n_label)
    b[-1]=np.inf
    loss=[]
    label=range(1,n_label+1)
    unique_y = np.sort(np.unique(y))
    y = np.array([np.where(unique_y == labels)[0][0] + 1 for labels in y]) 
    def predict(xw, b):
        scores = xw < b
        return np.argmax(scores) + 1
    for i in range(1,t+1):
        x=train[i-1]
        xw=(1+np.dot(x,w))**2
        y_bar=predict(xw, b)
        if y_bar !=y[i-1]:
            t_r=0
            for r in range(n_label - 1):
                if y_bar <= r:
                    yr = -1
                else:
                    yr = 1
                if yr * (xw - b[r]) <= 0:
                    t_r += yr
                    b[r] -= yr
            w=w+t_r*x
        #predicting score
        y_bar1=np.array([predict((1+np.dot(train[j-1],w))**2,b) for j in range(1,i+1)])
        l = np.mean(np.abs(y[:i] - y_bar1)) #loss function
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,w,b

In [6]:
def uniformmulticlassalgo(feature,y,maxiter):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    def taugenerate(h,e,yt):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        for k in e:
            t[k-1]=-(1/len(e))
        return t
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        e=[]
        for r in range(1,len(label)+1):
            if  r!=yt:
                if np.dot(m[int(r-1)],x)>=np.dot(m[int(yt-1)],x):
                    e.append(r)
        if len(e)!=0:
            tau=taugenerate(len(label),e,yt)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m         

In [7]:
def worstmulticlassalgo(feature,y,maxiter):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt,label):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        k=np.argsort([np.dot(m[j],x) for j in range(h)])[-1]
        t[k]=-1
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        e=[]
        for r in range(1,len(label)+1):
            if  r!=yt:
                if np.dot(m[int(r-1)],x)>=np.dot(m[int(yt-1)],x):
                    e.append(r)
        if len(e)!=0:
            tau=taugenerate(len(label),m,x,yt,label)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m

In [8]:
def vimulticlassalgo(feature,y,maxiter):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    y_bar1=[]
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        for i in range(h):
            if i!=int(yt-1):
                if np.dot(m[i],x)-np.dot(m[int(yt-1)],x)>0:
                    num=np.dot(m[i],x)-np.dot(m[int(yt-1)],x)
                else:
                    num=0
                den=0
                for k in range(h):
                    if np.dot(m[k],x)-np.dot(m[int(yt-1)],x)>0:
                        den+=np.dot(m[k],x)-np.dot(m[int(yt-1)],x)
                if den==0:
                    den=1
                t[i]=-(num/den)     
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        e=[]
        for r in range(1,len(label)+1):
            if  r!=yt:
                if np.dot(m[r-1],x)>=np.dot(m[int(yt-1)],x):
                    e.append(r)
        if len(e)!=0:
            tau=taugenerate(len(label),m,x,yt)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m

In [9]:
def mira(feature,y,maxiter):
    m=np.ones((len(np.unique(y)),len(feature[0])))
    y_bar1=[]
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt,ybart):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        tau=np.minimum(0.001, (((m[int(ybart-1)]-m[int(yt-1)])*x)+1.0)/(2.0*(np.dot(x,x))))
        t[int(yt-1)]=tau
        t[int(ybart-1)]=-tau
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        tau=taugenerate(len(label),m,x,yt,ybart)
        for k in label:
            m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m

In [10]:
w1_synthetic=[]
loss11=[]
for i in range(100):
    x=np.transpose(np.transpose(data[i])[:-1])
    y=np.transpose(np.transpose(data[i])[-1])
    loss1,w1=widrowhoff(x,y,7000,0.1)
    loss11.append(loss1)
    w1_synthetic.append(w1)

loss in round 1 is  4.0
loss in round 2 is  2.07276587484047
loss in round 3 is  2.2795666950460336
loss in round 4 is  1.9881005629680235
loss in round 5 is  1.6103009266819606
loss in round 6 is  1.528431139370358
loss in round 7 is  1.4459587476048639
loss in round 8 is  1.3999774360251902
loss in round 9 is  1.3395352707406967
loss in round 10 is  1.1754358586238969
loss in round 11 is  1.1734549924811672
loss in round 12 is  1.4264995492443873
loss in round 13 is  1.5463794348128816
loss in round 14 is  1.6143889312286301
loss in round 15 is  1.5383250791492107
loss in round 16 is  1.5555289086063777
loss in round 17 is  1.4874457844837417
loss in round 18 is  1.5760011845815525
loss in round 19 is  1.6263633549562169
loss in round 20 is  1.556556404414655
loss in round 21 is  1.627558757029246
loss in round 22 is  1.5978894672951105
loss in round 23 is  1.6306806081080598
loss in round 24 is  1.6288193759073841
loss in round 25 is  1.5155759219487452
loss in round 26 is  1.536658

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/synthetic/loss1.csv',loss11, delimiter=',')
np.savetxt('/home/22n0457/synthetic/w1_synthetic.csv',w1_synthetic, delimiter=',')

In [11]:
w2_synthetic=[]
b2_synthetic=[]
loss21=[]
#for i in range(100):
for i in range(1):
    x=np.transpose(np.transpose(data[i])[:-1])
    y=np.transpose(np.transpose(data[i])[-1])
    loss2,w2,b2=pranking(x,y,7000)
    loss21.append(loss2)
    w2_synthetic.append(w2)
    b2_synthetic.append(b2)

loss in round 1 is  1.0
loss in round 2 is  2.5
loss in round 3 is  2.3333333333333335
loss in round 4 is  2.5
loss in round 5 is  2.8
loss in round 6 is  2.8333333333333335
loss in round 7 is  2.857142857142857
loss in round 8 is  2.875
loss in round 9 is  2.888888888888889
loss in round 10 is  3.0
loss in round 11 is  3.0
loss in round 12 is  2.75
loss in round 13 is  2.6153846153846154
loss in round 14 is  2.5
loss in round 15 is  2.533333333333333
loss in round 16 is  2.5
loss in round 17 is  2.5294117647058822
loss in round 18 is  2.4444444444444446
loss in round 19 is  2.3684210526315788
loss in round 20 is  2.4
loss in round 21 is  2.3333333333333335
loss in round 22 is  2.272727272727273
loss in round 23 is  2.217391304347826
loss in round 24 is  2.125
loss in round 25 is  2.16
loss in round 26 is  2.1538461538461537
loss in round 27 is  2.185185185185185
loss in round 28 is  2.2142857142857144
loss in round 29 is  2.2758620689655173
loss in round 30 is  2.3
loss in round 31 is

KeyboardInterrupt: 

In [50]:
np.savetxt('/home/22n0457/synthetic/loss2.csv',loss21, delimiter=',')
np.savetxt('/home/22n0457/synthetic/w2_synthetic.csv',w2_synthetic, delimiter=',')
np.savetxt('/home/22n0457/synthetic/b2_synthetic.csv',b2_synthetic, delimiter=',')

In [12]:
loss31=[]
for i in range(100):
    x=np.transpose(np.transpose(data[i])[:-1])
    y=np.transpose(np.transpose(data[i])[-1])
    loss3,m1=uniformmulticlassalgo(x,y,7000)
    loss31.append(loss3)
    np.savetxt(f'/home/22n0457/synthetic/m1_synthetic___{i}.csv',m1, delimiter=',')

loss in round 1 is  0.0
loss in round 2 is  0.0
loss in round 3 is  0.6666666666666666
loss in round 4 is  1.0
loss in round 5 is  1.2000000000000002
loss in round 6 is  0.6666666666666666
loss in round 7 is  0.5714285714285714
loss in round 8 is  0.5
loss in round 9 is  0.4444444444444444
loss in round 10 is  0.4
loss in round 11 is  0.36363636363636365
loss in round 12 is  0.5833333333333333
loss in round 13 is  1.3076923076923077
loss in round 14 is  1.0714285714285714
loss in round 15 is  1.1333333333333333
loss in round 16 is  1.125
loss in round 17 is  1.0588235294117647
loss in round 18 is  1.3333333333333333
loss in round 19 is  1.4736842105263157
loss in round 20 is  0.9
loss in round 21 is  1.4285714285714284
loss in round 22 is  1.3636363636363638
loss in round 23 is  1.3043478260869565
loss in round 24 is  2.0
loss in round 25 is  1.08
loss in round 26 is  1.076923076923077
loss in round 27 is  1.037037037037037
loss in round 28 is  1.0
loss in round 29 is  1.0
loss in roun

KeyboardInterrupt: 

In [57]:
np.savetxt('/home/22n0457/synthetic/loss3.csv',loss31, delimiter=',')

In [13]:
loss41=[]
for i in range(100):
    x=np.transpose(np.transpose(data[i])[:-1])
    y=np.transpose(np.transpose(data[i])[-1])
    loss4,m2=worstmulticlassalgo(x,y,7000)
    loss41.append(loss4)
    np.savetxt(f'/home/22n0457/synthetic/m2_synthetic___{i}.csv',m2, delimiter=',')

loss in round 1 is  0.0
loss in round 2 is  1.5
loss in round 3 is  0.6666666666666666
loss in round 4 is  1.0
loss in round 5 is  1.0
loss in round 6 is  0.8333333333333333
loss in round 7 is  0.7142857142857142
loss in round 8 is  0.625
loss in round 9 is  0.5555555555555556
loss in round 10 is  0.6000000000000001
loss in round 11 is  0.5454545454545454
loss in round 12 is  0.8333333333333333
loss in round 13 is  0.9230769230769231
loss in round 14 is  1.0
loss in round 15 is  0.7333333333333333
loss in round 16 is  0.75
loss in round 17 is  0.8235294117647058
loss in round 18 is  1.0
loss in round 19 is  1.1052631578947367
loss in round 20 is  1.1
loss in round 21 is  1.1428571428571428
loss in round 22 is  1.0909090909090908
loss in round 23 is  1.0434782608695652
loss in round 24 is  1.5416666666666665
loss in round 25 is  1.48
loss in round 26 is  1.076923076923077
loss in round 27 is  1.037037037037037
loss in round 28 is  1.0
loss in round 29 is  1.4827586206896552
loss in roun

KeyboardInterrupt: 

In [13]:
np.savetxt('/home/22n0457/synthetic/loss4.csv',loss41, delimiter=',')

In [14]:
loss51=[]
for i in range(100):
    x=np.transpose(np.transpose(data[i])[:-1])
    y=np.transpose(np.transpose(data[i])[-1])
    loss5,m3=vimulticlassalgo(x,y,7000)
    loss51.append(loss5)
    np.savetxt(f'/home/22n0457/synthetic/m3_synthetic___{i}.csv',m3, delimiter=',')

loss in round 1 is  0.0
loss in round 2 is  1.5
loss in round 3 is  0.6666666666666666
loss in round 4 is  1.0
loss in round 5 is  1.0
loss in round 6 is  0.8333333333333333
loss in round 7 is  0.7142857142857142
loss in round 8 is  0.625
loss in round 9 is  0.5555555555555556
loss in round 10 is  0.6000000000000001
loss in round 11 is  0.5454545454545454
loss in round 12 is  0.75
loss in round 13 is  0.9230769230769231
loss in round 14 is  0.9285714285714285
loss in round 15 is  0.8
loss in round 16 is  0.875
loss in round 17 is  0.7647058823529411
loss in round 18 is  0.7777777777777777
loss in round 19 is  1.4736842105263157
loss in round 20 is  0.9
loss in round 21 is  1.1428571428571428
loss in round 22 is  1.0909090909090908
loss in round 23 is  1.0434782608695652
loss in round 24 is  2.125
loss in round 25 is  1.32
loss in round 26 is  1.4230769230769231
loss in round 27 is  1.3703703703703702
loss in round 28 is  1.3214285714285714
loss in round 29 is  1.2758620689655171
loss i

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/synthetic/loss5.csv',loss51, delimiter=',')

In [15]:
loss61=[]
for i in range(100):
    x=np.transpose(np.transpose(data[i])[:-1])
    y=np.transpose(np.transpose(data[i])[-1])
    loss6,m4=mira(x,y,7000)
    loss61.append(loss6)
    np.savetxt(f'/home/22n0457/synthetic/m4_synthetic___{i}.csv',m4, delimiter=',')

loss in round 1 is  0.0
loss in round 2 is  0.0
loss in round 3 is  0.6666666666666666
loss in round 4 is  1.0
loss in round 5 is  1.0
loss in round 6 is  2.833333333333333
loss in round 7 is  0.7142857142857142
loss in round 8 is  0.75
loss in round 9 is  1.1111111111111112
loss in round 10 is  2.0
loss in round 11 is  1.3636363636363638
loss in round 12 is  1.4166666666666665
loss in round 13 is  1.0
loss in round 14 is  1.357142857142857
loss in round 15 is  0.8666666666666667
loss in round 16 is  1.125
loss in round 17 is  0.8235294117647058
loss in round 18 is  1.1111111111111112
loss in round 19 is  1.4736842105263157
loss in round 20 is  1.0
loss in round 21 is  1.0476190476190474
loss in round 22 is  1.0454545454545454
loss in round 23 is  0.9565217391304348
loss in round 24 is  2.125
loss in round 25 is  1.08
loss in round 26 is  1.7307692307692308
loss in round 27 is  1.8888888888888888
loss in round 28 is  1.607142857142857
loss in round 29 is  2.1379310344827585
loss in rou

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/synthetic/loss6.csv',loss61, delimiter=',')